**Библиотеки**

In [1]:
import pandas as pd
import numpy as np
from utils import fetch_ohlcv_df, moex_fetch_ohlcv_df
import ccxt
from scipy.optimize import minimize

exchange = ccxt.binance({"enableRateLimit": True})

In [36]:
def Search_for_solutions(
    returns: pd.Series,
    cov: pd.DataFrame,
    risk: float | None = None,
    target_return: float | None = None
):
    tickers = returns.index
    r = returns.values.astype(float)
    C = cov.loc[tickers, tickers].values.astype(float)
    n = len(r)

    w0 = np.ones(n) / n
    bounds = [(0.0, 1.0)] * n
    cons = [{"type": "eq", "fun": lambda w: np.sum(w) - 1.0}]

    # 4) Минимальный риск при заданной доходности (target_return)
    if target_return is not None:
        cons.append({"type": "eq", "fun": lambda w: (w @ r) - target_return})
        obj = lambda w: (w.T @ C @ w)

    # 1) Минимальный риск (GMV)
    elif risk is None:
        obj = lambda w: (w.T @ C @ w)

    # 2) Макс. доходность при заданном риске (vol <= risk) или
    # 3) Макс. доходность без ограничения по риску (risk == 1)
    else:
        obj = lambda w: -(w @ r)
        if risk != 1:
            if risk <= 0:
                raise ValueError("risk должен быть > 0, либо risk=1 для max return без ограничений")
            cons.append({"type": "ineq", "fun": lambda w: risk - np.sqrt(max(w.T @ C @ w, 0.0))})

    res = minimize(obj, w0, method="SLSQP", bounds=bounds, constraints=cons,
                   options={"maxiter": 2000, "ftol": 1e-12})

    if not res.success:
        raise RuntimeError("Ограничения невыполнимы (risk слишком мал или target_return недостижим при long-only).")

    w = res.x
    w[np.abs(w) < 1e-10] = 0.0
    w = w / w.sum()

    var = float(w.T @ C @ w)
    return {
        "weights": pd.Series(w, index=tickers),
        "portfolio_risk": float(np.sqrt(max(var, 0.0))),
        "portfolio_return": float(w @ r),
    }

**Матрица доходностей**

In [18]:
ticker_list = ['BTC/USDT','ETH/USDT','TON/USDT','BNB/USDT','XRP/USDT','KAVA/USDT']

In [2]:
ticker_list = ['SBER','PLZL','MDMG','GAZP','VTBR','T','UGLD','NVTK','SELG']

In [20]:
df_pct_change = dict()

for ticker in ticker_list:
    df = fetch_ohlcv_df(exchange,ticker,'1M','2025-01-01','2025-12-31') #moex_fetch_ohlcv_df(ticker ,'1M','2025-01-01','2025-12-31') 
    df_pct_change[f'{ticker}'] = df['close'].pct_change().dropna()

In [21]:
df_pct_change = pd.DataFrame(df_pct_change)
df_pct_change

,BTC/USDT,ETH/USDT,TON/USDT,BNB/USDT,XRP/USDT,KAVA/USDT
timestamp,,,,,,
2025-02-01 00:00:00+00:00,-0.176508,-0.322146,-0.310545,-0.131962,-0.293403,-0.035738
2025-03-01 00:00:00+00:00,-0.021339,-0.185539,0.234675,0.028466,-0.025824,-0.002059
2025-04-01 00:00:00+00:00,0.140787,-0.015814,-0.230713,-0.008151,0.048280,-0.005044
2025-05-01 00:00:00+00:00,0.110647,0.409481,-0.004113,0.096903,-0.007760,-0.036636
2025-06-01 00:00:00+00:00,0.024425,-0.016847,-0.074968,-0.001687,0.028705,-0.043052
2025-07-01 00:00:00+00:00,0.080428,0.488004,0.211882,0.192175,0.351579,-0.049488
2025-08-01 00:00:00+00:00,-0.064940,0.187498,-0.103145,0.095143,-0.081624,-0.012096
2025-09-01 00:00:00+00:00,0.053605,-0.056168,-0.143760,0.175899,0.025615,-0.153846
2025-10-01 00:00:00+00:00,-0.038939,-0.071689,-0.157196,0.079712,-0.118414,-0.601132


**Ковариационная матрица**

In [7]:
cov = df_pct_change.cov(ddof = 1)
cov

,SBER,PLZL,MDMG,GAZP,VTBR,T,UGLD,NVTK,SELG
SBER,0.001115,-0.000857,0.000483,0.001923,0.002383,0.001167,-0.000084,0.002223,0.000315
PLZL,-0.000857,0.007115,0.003462,0.000925,-0.002894,0.001356,0.005200,0.002364,0.001202
MDMG,0.000483,0.003462,0.003602,0.002562,-0.002179,0.002138,0.002479,0.003649,0.002019
GAZP,0.001923,0.000925,0.002562,0.008408,0.005996,0.002103,-0.000046,0.008163,0.002004
VTBR,0.002383,-0.002894,-0.002179,0.005996,0.017447,0.000878,0.001199,0.004134,-0.001845
T,0.001167,0.001356,0.002138,0.002103,0.000878,0.002600,0.002649,0.003842,0.000625
UGLD,-0.000084,0.005200,0.002479,-0.000046,0.001199,0.002649,0.008064,0.002647,-0.000656
NVTK,0.002223,0.002364,0.003649,0.008163,0.004134,0.003842,0.002647,0.010661,0.001596
SELG,0.000315,0.001202,0.002019,0.002004,-0.001845,0.000625,-0.000656,0.001596,0.003279


**Риск и доходность(средняя по месяцу)**

In [8]:
std = df_pct_change.std(ddof = 1)
std

SBER    0.033397
PLZL    0.084352
MDMG    0.060020
GAZP    0.091693
VTBR    0.132088
T       0.050993
UGLD    0.089798
NVTK    0.103253
SELG    0.057259
dtype: float64

In [9]:
returns = df_pct_change.mean()
returns

SBER   -0.002676
PLZL    0.025132
MDMG    0.042289
GAZP   -0.026640
VTBR   -0.015344
T       0.000948
UGLD   -0.025102
NVTK   -0.003354
SELG   -0.015117
dtype: float64

In [10]:
#count = len(returns)
#w = np.ones(count) / count # Вектор распределения активов в портфеле (в данном случае равномерно)
w = np.array([0,0,1,0,0,0,0,0,0])
portfolio_returns = np.dot(w,returns) # Средняя доходность портфеля в МЕСЯЦ

portfolio_returns

np.float64(0.04228937305338194)

In [12]:
portfolio_var = w.T @ cov.values @ w
portfolio_risk = np.sqrt(portfolio_var)

portfolio_risk

np.float64(0.06002030794874591)

In [24]:
print(f'Риск - {portfolio_risk} \nОжидаемая доходность(за месяц) - {portfolio_returns}')

Риск - 0.06002030794874591 
Ожидаемая доходность(за месяц) - 0.04228937305338194


**ТЕСТЫ**

In [37]:
# 1. Минимальный риск
test_1 = Search_for_solutions(returns, cov)
test_1

{'weights': SBER    0.745220
 PLZL    0.175566
 MDMG    0.000000
 GAZP    0.000000
 VTBR    0.000000
 T       0.000000
 UGLD    0.000000
 NVTK    0.000000
 SELG    0.079214
 dtype: float64,
 'portfolio_risk': 0.026564820360281547,
 'portfolio_return': 0.0012207229119270795}

In [38]:
# Максимальная доходность при заданном риске
test_2 = Search_for_solutions(returns, cov, risk=0.035)
test_2

{'weights': SBER    0.520110
 PLZL    0.085891
 MDMG    0.380730
 GAZP    0.000000
 VTBR    0.013269
 T       0.000000
 UGLD    0.000000
 NVTK    0.000000
 SELG    0.000000
 dtype: float64,
 'portfolio_risk': 0.03500000000079551,
 'portfolio_return': 0.01666407695856636}

In [39]:
# 3. Максимальная доходность без ограничений по риску
test_3 = Search_for_solutions(returns, cov, risk = 1)
test_3

{'weights': SBER    0.0
 PLZL    0.0
 MDMG    1.0
 GAZP    0.0
 VTBR    0.0
 T       0.0
 UGLD    0.0
 NVTK    0.0
 SELG    0.0
 dtype: float64,
 'portfolio_risk': 0.06002030794874591,
 'portfolio_return': 0.04228937305338194}

In [ ]:
# 3. Минимальный риск при заданной доходности(в месяц)
test_4 = Search_for_solutions(returns,cov,target_return=0.03)
test_4

{'weights': SBER    0.165619
 PLZL    0.000000
 MDMG    0.750362
 GAZP    0.000000
 VTBR    0.084019
 T       0.000000
 UGLD    0.000000
 NVTK    0.000000
 SELG    0.000000
 dtype: float64,
 'portfolio_risk': 0.045757735207812816,
 'portfolio_return': 0.030000000000000002}